In [11]:
import cv2
import os
import numpy as np

input_img_folder = 'images'
input_ann_folder = 'labels'
output_ann_folder = 'Labels_new'
output_img_folder = 'Images_new'

os.makedirs(output_img_folder, exist_ok=True)
os.makedirs(output_ann_folder, exist_ok=True)

In [12]:
image_list = os.listdir(input_img_folder)
annotation_list = os.listdir(input_ann_folder)

label_dict = {
    "0" : "car",
    "1" : "truck",
    "2" : "bus",
    "3" : "minibus",
    "4" : "cyclist"
}

In [14]:
thickness = 2
color = (255,0,0)
count = 0

def object_string(label, bbox):
    req_str = '''
    <object>
        <name>{}</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{}</xmin>
            <ymin>{}</ymin>
            <xmax>{}</xmax>
            <ymax>{}</ymax>
        </bndbox>
    </object>
    '''.format(label, bbox[0], bbox[1], bbox[2], bbox[3])
    return req_str

for annotation in annotation_list:
    try:
        annotation_path = os.path.join(os.getcwd(), input_ann_folder, annotation)
        xml_annotation = annotation.split('.txt')[0] + '.xml'
        xml_path = os.path.join(os.getcwd(), output_ann_folder, xml_annotation)
        img_file = annotation.split('.txt')[0] + '.jpg'
        img_path = os.path.join(os.getcwd(), input_img_folder, img_file)
        output_img_path = os.path.join(os.getcwd(), output_img_folder, img_file)
        img = cv2.imread(img_path)
        annotation_string_init = '''
    <annotation>
        <folder>annotations</folder>
        <filename>{}</filename>
        <source>
            <database>Unknown</database>
        </source>
        <size>
            <width>{}</width>
            <height>{}</height>
            <depth>{}</depth>
        </size>
        <segmented>0</segmented>'''.format(img_file, img.shape[0], img.shape[1], img.shape[2])
    except:
        annotation_path = os.path.join(os.getcwd(), input_ann_folder, annotation)
        xml_annotation = annotation.split('.txt')[0] + '.xml'
        xml_path = os.path.join(os.getcwd(), output_ann_folder, xml_annotation)
        img_file = annotation.split('.txt')[0] + '.png'
        img_path = os.path.join(os.getcwd(), input_img_folder, img_file)
        output_img_path = os.path.join(os.getcwd(), output_img_folder, img_file)
        img = cv2.imread(img_path)
        annotation_string_init = '''
    <annotation>
        <folder>annotations</folder>
        <filename>{}</filename>
        <source>
            <database>Unknown</database>
        </source>
        <size>
            <width>{}</width>
            <height>{}</height>
            <depth>{}</depth>
        </size>
        <segmented>0</segmented>'''.format(img_file, img.shape[0], img.shape[1], img.shape[2])
        
    file = open(annotation_path, 'r')
    lines = file.readlines()
    for line in lines:
        new_line = line.strip('\n').split(' ')
        xmin = int(((float(new_line[1])*img.shape[1]+1)*2 - float(new_line[3])*img.shape[1])/2)
        xmax = int(((float(new_line[1])*img.shape[1]+1)*2 + float(new_line[3])*img.shape[1])/2)
        ymin = int(((float(new_line[2])*img.shape[0]+1)*2 - float(new_line[4])*img.shape[0])/2)
        ymax = int(((float(new_line[2])*img.shape[0]+1)*2 + float(new_line[4])*img.shape[0])/2)
        new_coords_min = (xmin, ymin)
        new_coords_max = (xmax, ymax)
        bbox = (xmin, ymin, xmax, ymax)
        label = label_dict.get(new_line[0])
        req_str = object_string(label, bbox)
        annotation_string_init = annotation_string_init + req_str
        #cv2.rectangle(img, new_coords_min, new_coords_max, color, thickness)
        #cv2.imshow('frame', cv2.resize(img, (400, 600)))
        #cv2.waitKey(0)
        #print(annotation_string_init)
    cv2.imwrite(output_img_path, img)
    annotation_string_final = annotation_string_init + '</annotation>'
    f = open(xml_path, 'w')
    f.write(annotation_string_final)
    f.close()
    count += 1
    print('[INFO] Completed {} image(s) and annotation(s) pair in {}'.format(count, img_file))

In [3]:
f = open('train/train.txt', 'w')
lines = os.listdir('train/images')
for line in lines:
    new_line = line.split('.jpg')[0].split('.png')[0]+'\n'
    f.write(new_line)
f.close()

In [4]:
f = open('val/val.txt', 'w')
lines = os.listdir('val/images')
for line in lines:
    new_line = line.split('.jpg')[0].split('.png')[0]+'\n'
    f.write(new_line)
f.close()